In [1]:
from google.colab import files

# Zip dosyası seç → yüklenir
uploaded = files.upload()


Saving Eggplant.zip to Eggplant.zip
Saving Eggplant_Tomato.zip to Eggplant_Tomato.zip
Saving Tomato.zip to Tomato.zip


In [2]:
import zipfile

# Tür (Domates/Patlıcan)
with zipfile.ZipFile("Eggplant_Tomato.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/Eggplant_Tomato")

# Patlıcan hastalıkları
with zipfile.ZipFile("Eggplant.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/Eggplant")

# Domates hastalıkları
with zipfile.ZipFile("Tomato.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/Tomato")


In [3]:
!ls /content/Eggplant_Tomato
!ls /content/Eggplant
!ls /content/Tomato


test  train  valid
test  train  valid
test  train  valid


In [4]:
from torchvision import datasets, transforms
import torch

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Tür modeli (Domates vs Patlıcan)
data_dir_species = "/content/Eggplant_Tomato"

image_datasets_species = {
    x: datasets.ImageFolder(root=f"{data_dir_species}/{x}", transform=transform)
    for x in ["train", "valid", "test"]
}

dataloaders_species = {
    x: torch.utils.data.DataLoader(image_datasets_species[x], batch_size=32, shuffle=True)
    for x in ["train", "valid", "test"]
}

class_species = image_datasets_species["train"].classes
print("📂 Tür sınıfları:", class_species)


📂 Tür sınıfları: ['Eggplant', 'Tomato']


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_data(data_dir, img_size=224, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    image_datasets = {x: datasets.ImageFolder(root=f"{data_dir}/{x}", transform=transform)
                      for x in ["train", "valid", "test"]}
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True)
                   for x in ["train", "valid", "test"]}
    class_names = image_datasets["train"].classes
    return dataloaders, class_names

def train_model(model, criterion, optimizer, dataloaders, num_epochs=50, patience=15):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    patience_counter = 0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        for phase in ["train", "valid"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss, running_corrects = 0.0, 0
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == "valid":
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter >= patience:
                        print("⏹️ Early stopping!")
                        model.load_state_dict(best_model_wts)
                        return model
    model.load_state_dict(best_model_wts)
    return model


In [6]:
# Tür dataset (Domates vs Patlıcan)
dataloaders_species, class_species = load_data("/content/Eggplant_Tomato")

model_species = models.resnet18(pretrained=True)
num_ftrs = model_species.fc.in_features
model_species.fc = nn.Linear(num_ftrs, len(class_species))
model_species = model_species.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_species.parameters(), lr=0.001)

print("🚀 Tür modeli eğitiliyor...")
model_species = train_model(model_species, criterion, optimizer, dataloaders_species)
torch.save(model_species.state_dict(), "model_species.pth")
print("✅ Tür modeli kaydedildi: model_species.pth")


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 216MB/s]


🚀 Tür modeli eğitiliyor...

Epoch 1/50
train Loss: 0.1247 Acc: 0.9456
valid Loss: 0.0260 Acc: 0.9919

Epoch 2/50
train Loss: 0.0201 Acc: 0.9936
valid Loss: 0.0452 Acc: 0.9839

Epoch 3/50
train Loss: 0.0031 Acc: 1.0000
valid Loss: 0.0097 Acc: 1.0000

Epoch 4/50
train Loss: 0.0116 Acc: 0.9952
valid Loss: 0.0153 Acc: 0.9919

Epoch 5/50
train Loss: 0.0153 Acc: 0.9904
valid Loss: 0.0156 Acc: 0.9919

Epoch 6/50
train Loss: 0.0116 Acc: 0.9952
valid Loss: 0.0135 Acc: 0.9919

Epoch 7/50
train Loss: 0.0005 Acc: 1.0000
valid Loss: 0.0052 Acc: 1.0000

Epoch 8/50
train Loss: 0.0004 Acc: 1.0000
valid Loss: 0.0040 Acc: 1.0000

Epoch 9/50
train Loss: 0.0002 Acc: 1.0000
valid Loss: 0.0024 Acc: 1.0000

Epoch 10/50
train Loss: 0.0000 Acc: 1.0000
valid Loss: 0.0012 Acc: 1.0000

Epoch 11/50
train Loss: 0.0000 Acc: 1.0000
valid Loss: 0.0012 Acc: 1.0000

Epoch 12/50
train Loss: 0.0000 Acc: 1.0000
valid Loss: 0.0015 Acc: 1.0000

Epoch 13/50
train Loss: 0.0000 Acc: 1.0000
valid Loss: 0.0014 Acc: 1.0000

Epoch 

In [7]:
# Patlıcan dataset
dataloaders_eggplant, class_eggplant = load_data("/content/Eggplant")

model_eggplant = models.resnet18(pretrained=True)
num_ftrs = model_eggplant.fc.in_features
model_eggplant.fc = nn.Linear(num_ftrs, len(class_eggplant))
model_eggplant = model_eggplant.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_eggplant.parameters(), lr=0.001)

print("🚀 Patlıcan hastalık modeli eğitiliyor...")
model_eggplant = train_model(model_eggplant, criterion, optimizer, dataloaders_eggplant)
torch.save(model_eggplant.state_dict(), "model_eggplant.pth")
print("✅ Patlıcan hastalık modeli kaydedildi: model_eggplant.pth")


🚀 Patlıcan hastalık modeli eğitiliyor...

Epoch 1/50
train Loss: 0.8966 Acc: 0.6887
valid Loss: 1.1975 Acc: 0.7573

Epoch 2/50
train Loss: 0.6034 Acc: 0.7963
valid Loss: 0.7453 Acc: 0.7524

Epoch 3/50
train Loss: 0.3281 Acc: 0.8875
valid Loss: 0.4488 Acc: 0.8689

Epoch 4/50
train Loss: 0.2566 Acc: 0.9185
valid Loss: 0.4278 Acc: 0.8786

Epoch 5/50
train Loss: 0.2405 Acc: 0.9263
valid Loss: 0.6705 Acc: 0.7427

Epoch 6/50
train Loss: 0.1303 Acc: 0.9564
valid Loss: 0.3374 Acc: 0.9175

Epoch 7/50
train Loss: 0.1068 Acc: 0.9631
valid Loss: 0.6661 Acc: 0.8592

Epoch 8/50
train Loss: 0.2107 Acc: 0.9311
valid Loss: 1.6201 Acc: 0.6165

Epoch 9/50
train Loss: 0.3302 Acc: 0.8914
valid Loss: 0.8567 Acc: 0.7767

Epoch 10/50
train Loss: 0.2266 Acc: 0.9292
valid Loss: 0.4142 Acc: 0.8544

Epoch 11/50
train Loss: 0.1751 Acc: 0.9486
valid Loss: 0.5614 Acc: 0.8010

Epoch 12/50
train Loss: 0.2668 Acc: 0.9146
valid Loss: 0.4220 Acc: 0.8689

Epoch 13/50
train Loss: 0.0729 Acc: 0.9787
valid Loss: 0.3659 Acc: 

In [8]:
# Domates dataset
dataloaders_tomato, class_tomato = load_data("/content/Tomato")

model_tomato = models.resnet18(pretrained=True)
num_ftrs = model_tomato.fc.in_features
model_tomato.fc = nn.Linear(num_ftrs, len(class_tomato))
model_tomato = model_tomato.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_tomato.parameters(), lr=0.001)

print("🚀 Domates hastalık modeli eğitiliyor...")
model_tomato = train_model(model_tomato, criterion, optimizer, dataloaders_tomato)
torch.save(model_tomato.state_dict(), "model_tomato.pth")
print("✅ Domates hastalık modeli kaydedildi: model_tomato.pth")


🚀 Domates hastalık modeli eğitiliyor...

Epoch 1/50
train Loss: 0.9370 Acc: 0.6625
valid Loss: 6.1533 Acc: 0.5097

Epoch 2/50
train Loss: 0.5041 Acc: 0.8312
valid Loss: 0.5707 Acc: 0.8155

Epoch 3/50
train Loss: 0.3270 Acc: 0.8894
valid Loss: 1.0693 Acc: 0.6893

Epoch 4/50
train Loss: 0.2606 Acc: 0.9127
valid Loss: 0.5095 Acc: 0.8447

Epoch 5/50
train Loss: 0.1421 Acc: 0.9573
valid Loss: 0.7872 Acc: 0.7427

Epoch 6/50
train Loss: 0.1314 Acc: 0.9525
valid Loss: 0.3825 Acc: 0.8883

Epoch 7/50
train Loss: 0.2413 Acc: 0.9117
valid Loss: 0.8185 Acc: 0.7767

Epoch 8/50
train Loss: 0.1693 Acc: 0.9476
valid Loss: 0.4405 Acc: 0.8447

Epoch 9/50
train Loss: 0.1485 Acc: 0.9554
valid Loss: 0.6695 Acc: 0.7961

Epoch 10/50
train Loss: 0.0673 Acc: 0.9816
valid Loss: 0.3996 Acc: 0.8835

Epoch 11/50
train Loss: 0.0392 Acc: 0.9903
valid Loss: 0.3882 Acc: 0.8738

Epoch 12/50
train Loss: 0.0509 Acc: 0.9855
valid Loss: 0.6539 Acc: 0.8252

Epoch 13/50
train Loss: 0.1238 Acc: 0.9661
valid Loss: 1.0543 Acc: 0

In [9]:
from google.colab import files
from PIL import Image
import torch
from torchvision import transforms

# Resim yükle
uploaded = files.upload()
img_path = list(uploaded.keys())[0]
print("👉 Yüklenen dosya:", img_path)


Saving Wilt-Disease-1-_jpg.rf.d60dae49221b75b0dab6fd49268301e0.jpg to Wilt-Disease-1-_jpg.rf.d60dae49221b75b0dab6fd49268301e0.jpg
👉 Yüklenen dosya: Wilt-Disease-1-_jpg.rf.d60dae49221b75b0dab6fd49268301e0.jpg


In [10]:
def predict_pipeline(img_path):
    # Resmi hazırla
    img = Image.open(img_path).convert("RGB")
    input_tensor = transform(img).unsqueeze(0).to(device)

    # 1. Tür tahmini
    with torch.no_grad():
        outputs = model_species(input_tensor)
        _, pred_species = torch.max(outputs, 1)

    if pred_species.item() == 0:  # Örn. 0 = Domates
        print("🌱 Tür: Domates")
        outputs = model_tomato(input_tensor)
        _, pred_disease = torch.max(outputs, 1)
        print("👉 Hastalık Tahmini:", class_tomato[pred_disease.item()])
    else:  # 1 = Patlıcan
        print("🌱 Tür: Patlıcan")
        outputs = model_eggplant(input_tensor)
        _, pred_disease = torch.max(outputs, 1)
        print("👉 Hastalık Tahmini:", class_eggplant[pred_disease.item()])


In [11]:
predict_pipeline(img_path)


🌱 Tür: Domates
👉 Hastalık Tahmini: Wilt


In [12]:
from google.colab import files
from PIL import Image
import torch

def upload_and_predict():
    uploaded = files.upload()
    for fn in uploaded.keys():
        img_path = fn
        print("\n📂 Dosya:", fn)
        predict_pipeline(img_path)   # Burada senin pipeline fonksiyonun çağrılıyor


In [13]:
upload_and_predict()


Saving Mosaic-Virus-157-_jpg.rf.88c9da2d99d709ab26538a5771945ec6.jpg to Mosaic-Virus-157-_jpg.rf.88c9da2d99d709ab26538a5771945ec6.jpg

📂 Dosya: Mosaic-Virus-157-_jpg.rf.88c9da2d99d709ab26538a5771945ec6.jpg
🌱 Tür: Domates
👉 Hastalık Tahmini: Mosaic Virus


In [14]:
upload_and_predict()


Saving Mosaic-Virus-36-_jpg.rf.2f1e6f86747a849578594574e96780c9.jpg to Mosaic-Virus-36-_jpg.rf.2f1e6f86747a849578594574e96780c9.jpg

📂 Dosya: Mosaic-Virus-36-_jpg.rf.2f1e6f86747a849578594574e96780c9.jpg
🌱 Tür: Domates
👉 Hastalık Tahmini: Mosaic Virus


In [15]:
upload_and_predict()

Saving t1.jpg to t1.jpg

📂 Dosya: t1.jpg
🌱 Tür: Domates
👉 Hastalık Tahmini: Wilt


In [16]:
upload_and_predict()

Saving t2.jpg to t2.jpg

📂 Dosya: t2.jpg
🌱 Tür: Domates
👉 Hastalık Tahmini: Wilt


In [17]:
upload_and_predict()

Saving p1.jpg to p1.jpg

📂 Dosya: p1.jpg
🌱 Tür: Domates
👉 Hastalık Tahmini: Wilt
